In [1]:
import pandas as pd
import json

from generate_graph import get_propositions, generateEdges, createGraph, get_propositions_nosplit
from main import query_graph, refine_query
from tqdm import tqdm
tqdm.pandas()

2024-11-07 00:34:45.217 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-11-07 00:34:45.423 
  command:

    streamlit run C:\Users\SLY\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [2]:
df = pd.read_json('datasets/HotPotQA/hotpot_dev_fullwiki_v1.json')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7405 entries, 0 to 7404
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               7405 non-null   object
 1   answer            7405 non-null   object
 2   question          7405 non-null   object
 3   supporting_facts  7405 non-null   object
 4   context           7405 non-null   object
 5   type              7405 non-null   object
 6   level             7405 non-null   object
dtypes: object(7)
memory usage: 405.1+ KB


In [4]:
df.head(5)

,_id,answer,question,supporting_facts,context,type,level
0,5a8b57f25542995d1e6f1371,yes,Were Scott Derrickson and Ed Wood of the same ...,"[[Scott Derrickson, 0], [Ed Wood, 0]]","[[Adam Collis, [Adam Collis is an American fil...",comparison,hard
1,5a8c7595554299585d9e36b6,Chief of Protocol,What government position was held by the woman...,"[[Kiss and Tell (1945 film), 0], [Shirley Temp...","[[A Kiss for Corliss, [A Kiss for Corliss is a...",bridge,hard
2,5a85ea095542994775f606a8,Animorphs,"What science fantasy young adult series, told ...","[[The Hork-Bajir Chronicles, 0], [The Hork-Baj...","[[Animorphs, [Animorphs is a science fantasy s...",bridge,hard
3,5adbf0a255429947ff17385a,no,Are the Laleli Mosque and Esma Sultan Mansion ...,"[[Laleli Mosque, 0], [Esma Sultan Mansion, 0]]","[[Esma Sultan, [Esma Sultan is the name of thr...",comparison,hard
4,5a8e3ea95542995a26add48d,"Greenwich Village, New York City","The director of the romantic comedy ""Big Stone...","[[Big Stone Gap (film), 0], [Adriana Trigiani,...","[[Great Eastern Conventions, [Great Eastern Co...",bridge,hard


In [5]:
df.iloc[0]['context']

[['Adam Collis',
  ['Adam Collis is an American filmmaker and actor.',
   ' He attended the Duke University from 1986 to 1990 and the University of California, Los Angeles from 2007 to 2010.',
   ' He also studied cinema at the University of Southern California from 1991 to 1997.',
   ' Collis first work was the assistant director for the Scott Derrickson\'s short "Love in the Ruins" (1995).',
   ' In 1998, he played "Crankshaft" in Eric Koyanagi\'s "Hundred Percent".']],
 ['Ed Wood (film)',
  ['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.',
   " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.",
   ' Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.']],
 ['Tyler Bates',
  ['Tyler Bates (born June 5, 1965) is a

In [6]:
df.iloc[0]['context']

[['Adam Collis',
  ['Adam Collis is an American filmmaker and actor.',
   ' He attended the Duke University from 1986 to 1990 and the University of California, Los Angeles from 2007 to 2010.',
   ' He also studied cinema at the University of Southern California from 1991 to 1997.',
   ' Collis first work was the assistant director for the Scott Derrickson\'s short "Love in the Ruins" (1995).',
   ' In 1998, he played "Crankshaft" in Eric Koyanagi\'s "Hundred Percent".']],
 ['Ed Wood (film)',
  ['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.',
   " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.",
   ' Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.']],
 ['Tyler Bates',
  ['Tyler Bates (born June 5, 1965) is a

In [7]:
df.iloc[0]['question']

'Were Scott Derrickson and Ed Wood of the same nationality?'

## Experiment


To run the evaluation script

```
python hotpot_evaluate_v1.py <path_to_prediction> <path_to_gold>
```

Example:

```
python hotpot_evaluate_v1.py <sample_dev_pred.json> <hotpot_dev_fullwiki_v1.json>
```

In [8]:
nested_list = df.iloc[0]['context']

In [9]:
nested_list[0][1]

['Adam Collis is an American filmmaker and actor.',
 ' He attended the Duke University from 1986 to 1990 and the University of California, Los Angeles from 2007 to 2010.',
 ' He also studied cinema at the University of Southern California from 1991 to 1997.',
 ' Collis first work was the assistant director for the Scott Derrickson\'s short "Love in the Ruins" (1995).',
 ' In 1998, he played "Crankshaft" in Eric Koyanagi\'s "Hundred Percent".']

In [10]:
nested_list[0:2]

[['Adam Collis',
  ['Adam Collis is an American filmmaker and actor.',
   ' He attended the Duke University from 1986 to 1990 and the University of California, Los Angeles from 2007 to 2010.',
   ' He also studied cinema at the University of Southern California from 1991 to 1997.',
   ' Collis first work was the assistant director for the Scott Derrickson\'s short "Love in the Ruins" (1995).',
   ' In 1998, he played "Crankshaft" in Eric Koyanagi\'s "Hundred Percent".']],
 ['Ed Wood (film)',
  ['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.',
   " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.",
   ' Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.']]]

In [11]:
propositions = []

for i, value in enumerate(tqdm(nested_list[0:2])):
        # print(nested_list[i][1])
        context = nested_list[i][1]
        context = " ".join(context)
        # print(context)
        get_propositions_nosplit(context, propositions)

100%|██████████| 2/2 [00:16<00:00,  8.27s/it]


In [18]:
# Open the file in read mode
with open('propositions.txt', 'r') as file:
    # Read all lines and store them in a list
    propositions_from_file = [line.strip() for line in file]

print(propositions_from_file)

['Adam Collis is an American filmmaker.', 'Adam Collis is an actor.', 'Adam Collis attended Duke University from 1986 to 1990.', 'Adam Collis attended the University of California, Los Angeles from 2007 to 2010.', 'Adam Collis studied cinema at the University of Southern California from 1991 to 1997.', "Adam Collis's first work was as the assistant director for Scott Derrickson's short 'Love in the Ruins' in 1995.", "In 1998, Adam Collis played 'Crankshaft' in Eric Koyanagi's film 'Hundred Percent'.", 'Ed Wood is a 1994 American biographical period comedy-drama film.', 'Tim Burton directed and produced Ed Wood.', 'Johnny Depp stars as cult filmmaker Ed Wood in Ed Wood.', "The film concerns the period in Ed Wood's life when Ed Wood made his best-known films.", "The film also concerns Ed Wood's relationship with actor Bela Lugosi.", 'Martin Landau plays the role of Bela Lugosi in Ed Wood.', 'Sarah Jessica Parker is among the supporting cast of Ed Wood.', 'Patricia Arquette is among the s

In [20]:
propositions_from_file

['Adam Collis is an American filmmaker.',
 'Adam Collis is an actor.',
 'Adam Collis attended Duke University from 1986 to 1990.',
 'Adam Collis attended the University of California, Los Angeles from 2007 to 2010.',
 'Adam Collis studied cinema at the University of Southern California from 1991 to 1997.',
 "Adam Collis's first work was as the assistant director for Scott Derrickson's short 'Love in the Ruins' in 1995.",
 "In 1998, Adam Collis played 'Crankshaft' in Eric Koyanagi's film 'Hundred Percent'.",
 'Ed Wood is a 1994 American biographical period comedy-drama film.',
 'Tim Burton directed and produced Ed Wood.',
 'Johnny Depp stars as cult filmmaker Ed Wood in Ed Wood.',
 "The film concerns the period in Ed Wood's life when Ed Wood made his best-known films.",
 "The film also concerns Ed Wood's relationship with actor Bela Lugosi.",
 'Martin Landau plays the role of Bela Lugosi in Ed Wood.',
 'Sarah Jessica Parker is among the supporting cast of Ed Wood.',
 'Patricia Arquette 

In [21]:
list_of_edges = generateEdges(propositions_from_file)

  0%|          | 0/18 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:44<00:00,  2.45s/it]


In [22]:
if createGraph(list_of_edges):
    print("Success")
else:
    print("Failed")

Success


In [13]:
# propositions = []

# for context in context_list:
#     get_propositions(context, propositions)

In [14]:
# # Define a function to apply to each row
# def print_qa(row):
    
#     question = row['question']
#     questionid = row['id']
#     result = query_graph(question)
    
    
#     if result is not None:
    
#         model_answer = result['result']

#         if "don't know the answer" in model_answer:
#             model_answer = ""
    
#         if len(row['answers']['text']) > 0:
#             real_answer = row['answers']['text'][0]
#         else:
#             real_answer = "\"\""

#         if len(result['intermediate_steps']) > 0:
#             cypher_query = result['intermediate_steps'][0]['query']
#             context = result['intermediate_steps'][1]['context']
       
#             if(len(context) < 1):
#             #     result = refine_query(cypher_query[6:], question)
#             #     model_answer = result['result'] 
#                 model_answer = "\"\""
#                 my_dict.update({questionid: ""})
#             else:
#                 my_dict.update({questionid: model_answer})
                
#         with open("preds.txt", "a") as preds:
#             preds.write("question: " + question + "\n")
#             preds.write("real_answer: " + real_answer + "\n")
#             preds.write("model_answer: " + model_answer + "\n")  
#             preds.write("cypher_query: " + cypher_query + "\n")
#             preds.write("======================" + "\n")  
#         preds.close()

#     else: 
#         my_dict.update({questionid: ""})
        